In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install tensorflow

In [ ]:
!pip install tflearn

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 497.5 MB 28 kB/s 
     |████████████████████████████████| 462 kB 65.4 MB/s 


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
import json
with open('/content/drive/MyDrive/simple_chat_bot/intents.json') as json_data:
  intents = json.load(json_data) #load data

In [ ]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['Tim is 5 Years old', 'My owner Saba is 20 years old!', '5 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Jim.', "I'm Jim!", "I'm Jim the assistant of Saba."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['what is the shops name', 'where is the shop?', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses

In [ ]:
import nltk
nltk.download('punkt') #download punkt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pickle

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
intents = json.loads(open('/content/drive/MyDrive/simple_chat_bot/intents.json').read())


In [ ]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
             
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('/content/drive/MyDrive/simple_chat_bot/words.pkl', 'wb'))
pickle.dump(classes, open('/content/drive/MyDrive/simple_chat_bot/classes.pkl', 'wb'))


In [ ]:
training = []
output_empty = [0] *len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov= True)
model.compile(loss= 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodelmodel.h5', hist)
print("Done")

Train on 30 samples


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
30/30 [==============================] - 2s 52ms/sample - loss: 1.9930 - acc: 0.1000
Epoch 2/200
30/30 [==============================] - 0s 548us/sample - loss: 1.9457 - acc: 0.1333
Epoch 3/200
30/30 [==============================] - 0s 541us/sample - loss: 1.9179 - acc: 0.2000
Epoch 4/200
30/30 [==============================] - 0s 516us/sample - loss: 1.8551 - acc: 0.3667
Epoch 5/200
30/30 [==============================] - 0s 547us/sample - loss: 1.7987 - acc: 0.4333
Epoch 6/200
30/30 [==============================] - 0s 540us/sample - loss: 1.7939 - acc: 0.3000
Epoch 7/200
30/30 [==============================] - 0s 539us/sample - loss: 1.7392 - acc: 0.4333
Epoch 8/200
30/30 [==============================] - 0s 593us/sample - loss: 1.6860 - acc: 0.5667
Epoch 9/200
30/30 [==============================] - 0s 529us/sample - loss: 1.6179 - acc: 0.4667
Epoch 10/200
30/30 [==============================] - 0s 553us/sample - loss: 1.5481 - acc: 0.6000
Epoch 11/200
30/30 [

In [ ]:
#chatbot.py
import random
import pickle 
import numpy as np
import json 

import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('/content/drive/MyDrive/simple_chat_bot/intents.json').read())

In [ ]:
words = pickle.load(open('/content/drive/MyDrive/simple_chat_bot/words.pkl', 'rb'))
classes = pickle.load(open('/content/drive/MyDrive/simple_chat_bot/classes.pkl', 'rb'))
model = load_model('/content/drive/MyDrive/simple_chat_bot/chatbot_model.model')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w: 
                bag[i] = 1
    return np.array(bag)


In [ ]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("GO! Bot is running!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)


GO! Bot is running!
hi


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Good to see you again!
what is your name
You can call me Jim.
your owner name is?
I'm Jim the assistant of Saba.
your age?
5 years young!
where is the shop?
We sell chocolate chip cookies for $2!
what is on the menu?
Cookies are on the menu!
shop name?
I'm Jim the assistant of Saba.
what is the shop name?
The shop is on 108 Backers Street
name of the sho?
I'm Jim the assistant of Saba.
name of the shop?
Cookies are on the menu!
what is the shop name?
The shop is on 108 Backers Street
what is the shops name
The shop is on 108 Backers Street
shop of the name?
Cookies are on the menu!
do you delivery?
Hello!
hi!
Hi there, how can I help?
give me cookies?
Hi there, how can I help?
